# Data Transformation from parquet

## 1. Import libraries

In [1]:
import pandas as pd

In [2]:
from pandas.api.types import CategoricalDtype

In [3]:
from datetime import datetime

## 2 Obtain the data in pickle

In [4]:
!#curl -O https://raw.githubusercontent.com/hkkenneth/covid-19-hk-data-extraction/master/raw-old-en.parquet

In [5]:
!#curl -O https://raw.githubusercontent.com/hkkenneth/covid-19-hk-data-extraction/master/raw-new-en.parquet

In [6]:
!#curl -O https://raw.githubusercontent.com/hkkenneth/covid-19-hk-data-extraction/master/raw-mode-en.parquet

In [7]:
!#curl -O https://raw.githubusercontent.com/hkkenneth/covid-19-hk-data-extraction/master/raw-cluster-en.parquet

In [8]:
!ls -lah raw*

-rw-r--r-- 1 root root 8.3K Aug 16 08:20 raw-cluster-en.parquet
-rw-r--r-- 1 root root 581K Aug 16 08:20 raw-en.dat
-rw-r--r-- 1 root root 3.1K Aug 16 08:20 raw-mode-en.parquet
-rw-r--r-- 1 root root 7.5K Aug 16 08:20 raw-new-en.parquet
-rw-r--r-- 1 root root  46K Aug 16 08:20 raw-old-en.parquet


In [9]:
hk_situation_en_old_df = pd.read_parquet('raw-old-en.parquet')

In [10]:
hk_situation_en_new_df = pd.read_parquet('raw-new-en.parquet')

In [11]:
hk_situation_en_cluster_df = pd.read_parquet('raw-cluster-en.parquet')

In [12]:
hk_situation_en_mode_df = pd.read_parquet('raw-mode-en.parquet')

## 3. Sanity check

In [13]:
type(hk_situation_en_new_df)

pandas.core.frame.DataFrame

In [14]:
hk_situation_en_new_df.shape

(48, 8)

In [15]:
hk_situation_en_new_df.head()

,Case no.,Report date,Date of onset,Gender,Age,HK/Non-HK resident,Case classification,Confirmed/probable
0,4314,14/08/2020,Asymptomatic,M,31,Unknown,Imported case,Confirmed
1,4315,14/08/2020,Asymptomatic,F,13,HK Resident,Epidemiologically linked with local case,Confirmed
2,4316,14/08/2020,03/08/2020,F,38,HK Resident,Local case,Confirmed
3,4317,14/08/2020,Asymptomatic,M,62,HK Resident,Epidemiologically linked with local case,Confirmed
4,4318,14/08/2020,31/07/2020,M,47,HK Resident,Local case,Confirmed


In [16]:
hk_situation_en_old_df.head()

,Case no.,Report date,Date of onset,Gender,Age,Hospitalised/Discharged/Deceased,HK/Non-HK resident,Case classification,Confirmed/probable
0,1,23/01/2020,21/01/2020,M,39,Discharged,Non-HK resident,I,Confirmed
1,2,23/01/2020,18/01/2020,M,56,Discharged,HK resident,I,Confirmed
2,3,24/01/2020,20/01/2020,F,62,Discharged,Non-HK resident,I,Confirmed
3,4,24/01/2020,23/01/2020,F,62,Discharged,Non-HK resident,I,Confirmed
4,5,24/01/2020,23/01/2020,M,63,Discharged,Non-HK resident,I,Confirmed


In [17]:
hk_situation_en_cluster_df.head()

,Cluster,Involved case number,Number of cases
0,Diamond Princess Cruise Ship,"14, 71, 78, 79, 80, 81, 87, 88, 94, 96, 97",11
1,Hotpot dinner gathering at Kwun Tong,"27, 29, 30, 31, 32, 33, 34, 35, 36, 37, 41, 44...",13
2,Fook Wai Ching Che in Maylun\rApartments in No...,"64, 65, 70, 73, 74, 76, 77, 83, 84, 86, 89, 91...",19
3,Travel tour to Egypt / Heng Tai House\rof Fu H...,"119, 120, 121, 123, 124, 125, 126, 127, 129, 140",10
4,Bar and band cluster,"182, 227, 276, 343, 360, 362, 367, 384, 386, 3...",103


In [18]:
hk_situation_en_mode_df.head()

,Category,Number of cases (percentage)
0,(1) Cases fulfilling the reporting criteria of...,336 (7.7%)
1,(2) Enhanced laboratory surveillance in public...,490 (11.2%)
2,(3) Enhanced surveillance at Accident and Emer...,1289 (29.6%)
3,(4) Diagnosis / Enhanced surveillance in priva...,624 (14.3%)
4,(5) Medical surveillance / contact tracing by ...,952 (21.8%)


## 4. Check & Unify Values

### 4.1 Resident Status

In [19]:
resident_mapping_casefold = {
  'hk resident': 'HK resident',
  'non-hk resident': 'Non-HK resident',
  'unknown': 'Unknown'
}

In [20]:
resident_type = CategoricalDtype(categories=[
                                             'HK resident',
                                             'Non-HK resident',
                                             'Unknown'
], ordered=True)

In [21]:
hk_situation_en_new_df['HK/Non-HK resident'].value_counts()

HK Resident    47
Unknown         1
Name: HK/Non-HK resident, dtype: int64

Ref: [Use casefold](https://stackoverflow.com/a/31599276)

In [22]:
hk_situation_en_new_df['HK/Non-HK resident'] = hk_situation_en_new_df['HK/Non-HK resident'].apply(str.casefold).map(resident_mapping_casefold).astype(resident_type)

In [23]:
hk_situation_en_new_df['HK/Non-HK resident'].value_counts()

HK resident        47
Unknown             1
Non-HK resident     0
Name: HK/Non-HK resident, dtype: int64

In [24]:
hk_situation_en_old_df['HK/Non-HK resident'].value_counts()

HK Resident        2817
HK resident        1377
Unknown              98
Non-HK resident      20
Non-HK Resident       1
Name: HK/Non-HK resident, dtype: int64

In [25]:
hk_situation_en_old_df['HK/Non-HK resident'] = hk_situation_en_old_df['HK/Non-HK resident'].apply(str.casefold).map(resident_mapping_casefold).astype(resident_type)

In [26]:
hk_situation_en_old_df['HK/Non-HK resident'].value_counts()

HK resident        4194
Unknown              98
Non-HK resident      21
Name: HK/Non-HK resident, dtype: int64

### 4.2 Case Classification

In [27]:
classification_mapping = {
  'i': 'Imported case',
  'l': 'Local case',
  'pl': 'Possibly local case',
  'epi-i': 'Epidemiologically linked with imported case',
  'epi-l': 'Epidemiologically linked with local case',
  'epi-pl': 'Epidemiologically linked with possibly local case'
}

classification_mapping_casefold = {
  'imported case': 'Imported case',
  'local case': 'Local case',
  'possibly local case': 'Possibly local case',
  'epidemiologically linked with imported case': 'Epidemiologically linked with imported case',
  'epidemiologically linked with local case': 'Epidemiologically linked with local case',
  'epidemiologically linked with possibly local case': 'Epidemiologically linked with possibly local case'
}

In [28]:
classification_type = CategoricalDtype(categories=[
                                                   'Local case',
                                                   'Possibly local case',
                                                   'Imported case',
                                                   'Epidemiologically linked with local case',
                                                   'Epidemiologically linked with possibly local case',
                                                   'Epidemiologically linked with imported case'
], ordered=True)

In [29]:
hk_situation_en_new_df['Case classification'] = hk_situation_en_new_df['Case classification'].apply(str.casefold).map(classification_mapping_casefold).fillna(hk_situation_en_new_df['Case classification'])

In [30]:
hk_situation_en_new_df['Case classification'].value_counts()

Epidemiologically linked with local case    35
Local case                                  11
Imported case                                2
Name: Case classification, dtype: int64

Ref: [map can be much faster than replace](https://stackoverflow.com/a/41678874)

In [31]:
hk_situation_en_old_df['Case classification'] = hk_situation_en_old_df['Case classification'].apply(str.casefold).map(classification_mapping).fillna(hk_situation_en_old_df['Case classification'])

In [32]:
hk_situation_en_old_df['Case classification'].value_counts()

Epidemiologically linked with local case             1844
Local case                                           1189
Imported case                                        1084
Possibly local case                                   103
Epidemiologically linked with possibly local case      62
Epidemiologically linked with imported case            31
Name: Case classification, dtype: int64

In [33]:
hk_situation_en_old_df['Case classification'] = hk_situation_en_old_df['Case classification'].astype(classification_type)

In [34]:
hk_situation_en_new_df['Case classification'] = hk_situation_en_new_df['Case classification'].astype(classification_type)

### 4.3 Gender

In [35]:
gender_mapping_casefold = {
  'f': 'Female',
  'm': 'Male',
  'unknown': 'Unknown'
}

In [36]:
gender_type = CategoricalDtype(categories=[
                                           'Female',
                                           'Male',
                                           'Unknown'
], ordered=True)

In [37]:
hk_situation_en_old_df['Gender'] = hk_situation_en_old_df['Gender'].apply(str.casefold).map(gender_mapping_casefold).astype(gender_type)

In [38]:
hk_situation_en_new_df['Gender'] = hk_situation_en_new_df['Gender'].apply(str.casefold).map(gender_mapping_casefold).astype(gender_type)

In [39]:
hk_situation_en_old_df['Gender'].value_counts()

Female     2171
Male       2142
Unknown       0
Name: Gender, dtype: int64

In [40]:
hk_situation_en_new_df['Gender'].value_counts()

Male       24
Female     24
Unknown     0
Name: Gender, dtype: int64

### 4.4 Age

In [41]:
hk_situation_en_old_df['Age'] = hk_situation_en_old_df['Age'].apply(lambda a: (float(a.split('\r')[0]) / 12.0) if '\rmonth' in a else a)

In [42]:
hk_situation_en_old_df['Age'] = pd.to_numeric(hk_situation_en_old_df['Age'], errors='coerce')

In [43]:
hk_situation_en_new_df['Age'] = hk_situation_en_new_df['Age'].apply(lambda a: (float(a.split('\r')[0]) / 12.0) if '\rmonth' in a else a)

In [44]:
hk_situation_en_new_df['Age'] = pd.to_numeric(hk_situation_en_new_df['Age'], errors='coerce')

### 4.5 Hospitalised / Discharged / Deceased Status

In [45]:
status_mapping_casefold = {
  'deceased': 'Deceased',
  'discharged': 'Discharged',
  'hospitalised': 'Hospitalised',
  'new': 'New',
  'no admission': 'No admission'
}

In [46]:
status_type = CategoricalDtype(categories=[
                                           'Hospitalised',
                                           'Discharged',
                                           'Deceased',
                                           'New',
                                           'No admission'
], ordered=True)

In [47]:
hk_situation_en_old_df['Hospitalised/Discharged/Deceased'].value_counts()

Discharged      3392
Hospitalised     829
Deceased          66
No admission      26
Name: Hospitalised/Discharged/Deceased, dtype: int64

In [48]:
hk_situation_en_old_df['Hospitalised/Discharged/Deceased'] = hk_situation_en_old_df['Hospitalised/Discharged/Deceased'].apply(str.casefold).map(status_mapping_casefold).astype(status_type)

In [49]:
hk_situation_en_old_df['Hospitalised/Discharged/Deceased'].value_counts()

Discharged      3392
Hospitalised     829
Deceased          66
No admission      26
New                0
Name: Hospitalised/Discharged/Deceased, dtype: int64

### 4.6 Confirmed / probable status

In [50]:
confirmation_status_mapping_casefold = {
  'confirmed': 'Confirmed',
  'probable': 'Probable'
}

In [51]:
confirmation_status_type = CategoricalDtype(categories=[
                                           'Confirmed',
                                           'Probable'
], ordered=True)

In [52]:
hk_situation_en_old_df['Confirmed/probable'] = hk_situation_en_old_df['Confirmed/probable'].apply(str.casefold).map(confirmation_status_mapping_casefold).astype(confirmation_status_type)

In [53]:
hk_situation_en_old_df['Confirmed/probable'].value_counts()

Confirmed    4312
Probable        1
Name: Confirmed/probable, dtype: int64

In [54]:
hk_situation_en_new_df['Confirmed/probable'] = hk_situation_en_new_df['Confirmed/probable'].apply(str.casefold).map(confirmation_status_mapping_casefold).astype(confirmation_status_type)

In [55]:
hk_situation_en_new_df['Confirmed/probable'].value_counts()

Confirmed    48
Probable      0
Name: Confirmed/probable, dtype: int64

### 4.7 Report date

In [56]:
hk_situation_en_new_df['Report date'] = pd.to_datetime(hk_situation_en_new_df['Report date'], format='%d/%m/%Y')

In [57]:
# Did not use
# hk_situation_en_new_df['Report date'] = hk_situation_en_new_df['Report date'].apply(lambda d: datetime.strptime(d, '%d/%m/%Y').date()).

In [58]:
hk_situation_en_old_df['Report date'] = pd.to_datetime(hk_situation_en_old_df['Report date'], format='%d/%m/%Y')

### 4.8 Onset Date

Check what we may see

In [59]:
hk_situation_en_new_df['Date of onset'].value_counts()

Asymptomatic       24
11/08/2020          7
12/08/2020          3
07/08/2020          3
10/08/2020          2
08/08/2020          2
30/07/2020          1
04/08/2020          1
31/07/2020          1
09/08/2020          1
03/08/2020          1
06/08/2020          1
Mid- July, 2020     1
Name: Date of onset, dtype: int64

In [60]:
hk_situation_en_old_df['Date of onset'].value_counts()

Asymptomatic    915
27/07/2020      131
25/07/2020      118
24/07/2020      109
21/07/2020      106
               ... 
29/05/2020        1
18/01/2020        1
05/05/2020        1
12/05/2020        1
25/05/2020        1
Name: Date of onset, Length: 168, dtype: int64

In [61]:
hk_situation_en_new_df['Date of onset original'] = hk_situation_en_new_df['Date of onset'].astype(str)

In [62]:
hk_situation_en_old_df['Date of onset original'] = hk_situation_en_old_df['Date of onset'].astype(str)

In [63]:
hk_situation_en_new_df['Date of onset'] = pd.to_datetime(hk_situation_en_new_df['Date of onset original'], format='%d/%m/%Y', errors='coerce')

In [64]:
hk_situation_en_old_df['Date of onset'] = pd.to_datetime(hk_situation_en_old_df['Date of onset original'], format='%d/%m/%Y', errors='coerce')

In [65]:
hk_situation_en_new_df['Date of onset']

0           NaT
1           NaT
2    2020-08-03
3           NaT
4    2020-07-31
5    2020-08-11
6           NaT
7           NaT
8           NaT
9           NaT
10          NaT
11          NaT
12          NaT
13   2020-08-07
14   2020-08-11
15   2020-08-06
0    2020-08-10
1    2020-08-08
2    2020-08-11
3    2020-08-12
4    2020-08-11
5           NaT
6    2020-08-07
7    2020-08-04
8    2020-08-11
9    2020-08-09
10          NaT
11   2020-08-07
12          NaT
13          NaT
14   2020-08-12
15   2020-08-11
16          NaT
17   2020-08-12
18          NaT
19          NaT
20          NaT
21          NaT
22   2020-08-10
0           NaT
1           NaT
2           NaT
3           NaT
4    2020-07-30
5    2020-08-08
6    2020-08-11
7           NaT
8           NaT
Name: Date of onset, dtype: datetime64[ns]

TODO: for Asymptomatic, a flag should be added to data engineering stage

TODO: for other malformatted date, should use a flag to indicate and manually fix with code (reproducible)

In [66]:
hk_situation_en_new_df['Date of onset original']

0        Asymptomatic
1        Asymptomatic
2          03/08/2020
3        Asymptomatic
4          31/07/2020
5          11/08/2020
6        Asymptomatic
7        Asymptomatic
8        Asymptomatic
9        Asymptomatic
10       Asymptomatic
11       Asymptomatic
12       Asymptomatic
13         07/08/2020
14         11/08/2020
15         06/08/2020
0          10/08/2020
1          08/08/2020
2          11/08/2020
3          12/08/2020
4          11/08/2020
5     Mid- July, 2020
6          07/08/2020
7          04/08/2020
8          11/08/2020
9          09/08/2020
10       Asymptomatic
11         07/08/2020
12       Asymptomatic
13       Asymptomatic
14         12/08/2020
15         11/08/2020
16       Asymptomatic
17         12/08/2020
18       Asymptomatic
19       Asymptomatic
20       Asymptomatic
21       Asymptomatic
22         10/08/2020
0        Asymptomatic
1        Asymptomatic
2        Asymptomatic
3        Asymptomatic
4          30/07/2020
5          08/08/2020
6         

In [67]:
hk_situation_en_new_df['Date of onset original'][hk_situation_en_new_df['Date of onset'].isnull()]

0        Asymptomatic
1        Asymptomatic
3        Asymptomatic
6        Asymptomatic
7        Asymptomatic
8        Asymptomatic
9        Asymptomatic
10       Asymptomatic
11       Asymptomatic
12       Asymptomatic
5     Mid- July, 2020
10       Asymptomatic
12       Asymptomatic
13       Asymptomatic
16       Asymptomatic
18       Asymptomatic
19       Asymptomatic
20       Asymptomatic
21       Asymptomatic
0        Asymptomatic
1        Asymptomatic
2        Asymptomatic
3        Asymptomatic
7        Asymptomatic
8        Asymptomatic
Name: Date of onset original, dtype: object

TODO fix the index

In [68]:
hk_situation_en_old_df['Date of onset original'][hk_situation_en_old_df['Date of onset'].isnull()].value_counts()

Asymptomatic    915
Pending          59
Unknown          23
January           1
Mid-March         1
Name: Date of onset original, dtype: int64

In [69]:
hk_situation_en_new_df.dtypes

Case no.                           int64
Report date               datetime64[ns]
Date of onset             datetime64[ns]
Gender                          category
Age                                int64
HK/Non-HK resident              category
Case classification             category
Confirmed/probable              category
Date of onset original            object
dtype: object

In [70]:
hk_situation_en_new_df.index

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,  0,
             1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
            18, 19, 20, 21, 22,  0,  1,  2,  3,  4,  5,  6,  7,  8],
           dtype='int64')

In [71]:
hk_situation_en_new_df = hk_situation_en_new_df.reset_index(drop=True)

In [72]:
hk_situation_en_new_df.index

RangeIndex(start=0, stop=48, step=1)

In [73]:
hk_situation_en_old_df = hk_situation_en_old_df.reset_index(drop=True)

In [74]:
hk_situation_en_old_df.index

RangeIndex(start=0, stop=4313, step=1)

## 5. Merge new and old

In [75]:
hk_situation_en_old_df.columns

Index(['Case no.', 'Report date', 'Date of onset', 'Gender', 'Age',
       'Hospitalised/Discharged/Deceased', 'HK/Non-HK resident',
       'Case classification', 'Confirmed/probable', 'Date of onset original'],
      dtype='object')

In [76]:
hk_situation_en_new_df.columns

Index(['Case no.', 'Report date', 'Date of onset', 'Gender', 'Age',
       'HK/Non-HK resident', 'Case classification', 'Confirmed/probable',
       'Date of onset original'],
      dtype='object')

In [77]:
human_merged_name = ['Case no.', 'Report date', 'Date of onset', 'Gender', 'Age', 'Status', 'Resident status', 'Case classification', 'Confirmation status', 'Date of onset original']

In [78]:
# Ref: https://stackoverflow.com/a/44296858
columns_rename_dict = dict(zip(hk_situation_en_old_df.columns, human_merged_name))
hk_situation_en_old_df = hk_situation_en_old_df.rename(columns=columns_rename_dict)

In [79]:
hk_situation_en_old_df.dtypes

Case no.                           int64
Report date               datetime64[ns]
Date of onset             datetime64[ns]
Gender                          category
Age                              float64
Status                          category
Resident status                 category
Case classification             category
Confirmation status             category
Date of onset original            object
dtype: object

In [80]:
hk_situation_en_new_df = hk_situation_en_new_df.rename(columns=columns_rename_dict)

In [81]:
hk_situation_en_new_df['Status'] = 'New'

In [82]:
hk_situation_en_new_df['Status'] = hk_situation_en_new_df['Status'].astype(status_type)

In [83]:
hk_situation_en_new_df['Status'].value_counts()

New             48
No admission     0
Deceased         0
Discharged       0
Hospitalised     0
Name: Status, dtype: int64

In [84]:
hk_situation_en_merged_df = pd.concat([hk_situation_en_old_df, hk_situation_en_new_df])

In [85]:
hk_situation_en_merged_df.dtypes

Case no.                           int64
Report date               datetime64[ns]
Date of onset             datetime64[ns]
Gender                          category
Age                              float64
Status                          category
Resident status                 category
Case classification             category
Confirmation status             category
Date of onset original            object
dtype: object

In [86]:
hk_situation_en_merged_df['Status'].value_counts()

Discharged      3392
Hospitalised     829
Deceased          66
New               48
No admission      26
Name: Status, dtype: int64

## 6. Export to parquet

In [87]:
hk_situation_en_new_df.to_parquet('transformed-new-en.parquet')

In [88]:
hk_situation_en_old_df.to_parquet('transformed-old-en.parquet')

In [89]:
hk_situation_en_merged_df.to_parquet('transformed-merged-en.parquet')